### Eksperymenty

**Modele**
- resnet50
- vgg13

**Zbiory danych**
- CIFAR100
- SVHN

**Metody**
- KMeans
- KMeansPurity
- KMeansDino
- KMeansDinoPurity
- Spectral (?)
- SpectralPurity (?)

**Ustawienia**
- ratio = (0.1, 1, 10)
- epochs = 10
- batch_size = 512
- clip = 5.0
- num_clusters = 20
- eqsize = True
- min_purity = 0.1 (SVHN), 0.01 (CIFAR100)

**Nazewnictwo przebiegów**
W nawiasach kwadratowych podano wartość do wyboru ("jeden z ...") w nawiasch klamrowych wartości opcjonalne.
ID służy do rozróżniania eksperymentów w tej samej konfiguracji

- [r50,vgg13],ep:10,bs:512,clip:5.0,{nc:20},{eqsize},{mp:[0.1,0.01]},ID:[1,2,3,..]
- przykład: r50,ep:10,bs:512,clip:5.0,ID:1
- przykład: r50,ep:10,bs:512,clip:50,nc:20,eqsize,mp:0.1,ID:1

**Uwagi**
- Przy każdym eksperymencie nowy model (!)